In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2

In [ ]:
def canny_edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

In [ ]:
data_dir = '/path/to/your/Finger-Nails-Dataset'

batch_size = 32
epochs = 40
learning_rate = 0.001
gamma = 0.5

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,
    preprocessing_function=canny_edge_detection,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_test_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

val_gen, test_gen = train_test_split(list(val_test_gen), test_size=0.333, random_state=42)

def list_to_gen(img_list, batch_size, target_size=(224, 224)):
    img_array = np.array([img for img, label in img_list])
    labels = np.array([label for img, label in img_list])
    datagen = ImageDataGenerator()
    gen = datagen.flow(img_array, labels, batch_size=batch_size)
    return gen

val_gen = list_to_gen(val_gen, batch_size)
test_gen = list_to_gen(test_gen, batch_size)

In [ ]:
# Model setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=gamma, patience=5, min_lr=1e-6)

history = model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    callbacks=[reduce_lr]
)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history_finetune.history['accuracy'], label='Finetune Training Accuracy')
plt.plot(history_finetune.history['val_accuracy'], label='Finetune Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history_finetune.history['loss'], label='Finetune Training Loss')
plt.plot(history_finetune.history['val_loss'], label='Finetune Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
test_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='binary',
    subset='validation'
)

test_loss, test_accuracy = model.evaluate(test_gen)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')

# Predict on the test set
test_gen.reset()
y_true = test_gen.classes
y_pred = model.predict(test_gen)
y_pred = np.where(y_pred > 0.5, 1, 0).flatten()

# Calculate additional metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Matthews Correlation Coefficient (MCC): {mcc}')